In [1]:
import numpy as np
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import os.path

In [2]:
if os.path.exists('LC_gwr.csv'):
    ts = pd.read_csv('LC_gwr.csv')
else:
    lc = pd.read_csv('../LC/Lending_Club_Accepted_2014_2018.csv')
    lc['issue_d'] = pd.to_datetime(lc['issue_d'], format="%b-%Y")
    lc['issue_d_str'] = lc['issue_d'].apply(lambda x: x.strftime('%Y-%m-%d'))

    ind = []
    for x in lc['issue_d_str'].unique():
        for y in lc['addr_state'].unique():
            ind.append(str(x) + ";" + str(y))
    print(len(ind))

    ts = pd.DataFrame()

    ts['ind'] = ind
    ts[['date', 'addr_state']] = ts['ind'].str.split(';', 1, expand=True)
    print('Processed date, addr_state')


    ts['total_funded'] = ts['ind'].apply(lambda x: sum(lc['funded_amnt'][
            (lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1])
        ]))
    print('Processed total_funded')

    ts['payback_success_rate'] = ts['ind'].apply(lambda x: len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Fully Paid')])
                                               / (len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Charged Off')]) + 
                                                  len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Fully Paid')]))
                                                 if (len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Charged Off')]) + 
                                                  len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Fully Paid')]))
                                                 else 0
                                                )
    print('Processed payback_success_rate')

    ts['payback_failure_rate'] = ts['ind'].apply(lambda x: len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Charged Off')])
                                               / (len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Charged Off')]) + 
                                                  len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Fully Paid')]))
                                                 if (len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Charged Off')]) + 
                                                  len(lc[(lc['issue_d_str'] == x.split(';')[0]) & (lc['addr_state'] == x.split(';')[1]) & (lc['loan_status'] == 'Fully Paid')]))
                                                 else 0
                                                )
    print('Processed payback_failure_rate')

    ts['date'] = pd.to_datetime(ts['date'], format="%Y-%m-%d")
    ts = ts[ts['date'] < pd.to_datetime('06-2017', format='%m-%Y')]
    ts.to_csv('LC_gwr.csv', index=False)

ts.head()

,ind,date,addr_state,total_funded,payback_success_rate
0,2015-12-01;PA,2015-12-01,PA,22969425.0,0.781983
1,2015-12-01;SD,2015-12-01,SD,1294050.0,0.775000
2,2015-12-01;IL,2015-12-01,IL,26084475.0,0.818120
3,2015-12-01;NJ,2015-12-01,NJ,26159675.0,0.775972
4,2015-12-01;GA,2015-12-01,GA,23689500.0,0.827381


In [12]:
ec = pd.read_csv('Macroeconomic Data.csv')
ec['date'] = pd.to_datetime(ec['Month-Year'], format="%m-%Y")
ec['date'] = ec['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

del ec['Month-Year']
ec.head()

,CPI,FFR,UR,IR,MR,date
0,233.916,0.07,6.6,1.6,4.53,2014-01-01
1,234.781,0.07,6.7,1.1,4.23,2014-02-01
2,236.293,0.06,6.7,1.5,4.28,2014-03-01
3,237.072,0.08,6.2,2.0,4.41,2014-04-01
4,237.900,0.09,6.3,2.1,4.29,2014-05-01


In [65]:
df = pd.merge(ts, ec, how='left', on='date')
df.head()

,ind,date,addr_state,total_funded,payback_success_rate,lat,lon,x,y,CPI,FFR,UR,IR,MR
0,2015-12-01;PA,2015-12-01,PA,22969425.0,0.781983,8.559559,-81.130843,-9.031444e+06,9.564099e+05,236.525,0.13,5.0,0.7,3.93
1,2015-12-01;SD,2015-12-01,SD,1294050.0,0.775000,14.584444,29.491769,3.283009e+06,1.641355e+06,236.525,0.13,5.0,0.7,3.93
2,2015-12-01;IL,2015-12-01,IL,26084475.0,0.818120,40.079661,-89.433729,-9.955717e+06,4.877525e+06,236.525,0.13,5.0,0.7,3.93
3,2015-12-01;NJ,2015-12-01,NJ,26159675.0,0.775972,40.075738,-74.404162,-8.282633e+06,4.876954e+06,236.525,0.13,5.0,0.7,3.93
4,2015-12-01;GA,2015-12-01,GA,23689500.0,0.827381,-0.899969,11.689970,1.301321e+06,-1.001883e+05,236.525,0.13,5.0,0.7,3.93


In [140]:
# Prepare dataset inputs
ec_fts = ['CPI', 'FFR', 'UR', 'IR', 'MR']
g_y = df['payback_success_rate'].values.reshape((-1,1))
g_X = df[ec_fts].values
u = df['x']
v = df['y']

g_coords = list(zip(u,v))

g_X = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0)

g_y = g_y.reshape((-1,1))

g_y = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)

In [67]:
# Calibrate GWR model

gwr_selector = Sel_BW(g_coords, g_y, g_X)
gwr_bw = gwr_selector.search(bw_min=2)
print(gwr_bw)
gwr_results = GWR(g_coords, g_y, g_X, gwr_bw).fit()

47.0


In [68]:
gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                2091
Number of covariates:                                                     6

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                           2087.397
Log-likelihood:                                                   -2965.197
AIC:                                                               5942.395
AICc:                                                              5944.449
BIC:                                                             -13853.257
R2:                                                                   0.002
Adj. R2:                                                             -0.001

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [126]:
def mean(lst):
    return sum(lst)/len(lst)

states_ind = df['addr_state']
coeffs = pd.DataFrame()
coeffs.index = states_ind.unique()
for i in range(len(ec_fts)):
    col = ec_fts[i]
    coeffs[col] = [[] for _ in range(len(coeffs))]
    for j in range(len(gwr_results.params[:, i + 1])):
        coeffs.at[states_ind[j], col].append(gwr_results.params[:, i + 1][j])
    for j in range(len(coeffs)):
        coeffs.loc[states_ind[j]][col] = mean(coeffs.loc[states_ind[j]][col])

for col in coeffs.columns:
    coeffs[col] = coeffs[col].astype(float)
    
coeffs.head()

,CPI,FFR,UR,IR,MR
PA,-0.079606,0.030946,0.021600,-0.022325,0.075872
SD,-0.119476,0.097245,0.114646,0.102230,0.020214
IL,-0.039761,-0.027163,0.038570,-0.018780,0.069159
NJ,-0.077144,-0.024428,0.034034,-0.001591,0.065205
GA,-0.071765,-0.012803,-0.039564,-0.007547,0.106716


In [152]:
import plotly.express as px
from urllib.request import urlopen
import json

with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA.geo.json') as response:
    states = json.load(response)

coeffs['FFR Coefficient'] = coeffs['FFR']
col = 'FFR Coefficient'
fig = px.choropleth(coeffs.reset_index()[['index', col]], locations='index', color=col,
                           locationmode="USA-states",
                            color_continuous_scale=px.colors.diverging.RdYlBu,
                           scope="usa",
                            range_color=[-0.1, 0.1])

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [149]:
coeffs.sort_values('FFR').head()

,CPI,FFR,UR,IR,MR
MS,0.167103,-0.949924,-0.924813,0.281400,-0.209294
WV,-0.721049,-0.404478,0.132107,-0.496913,0.274441
ND,-0.124376,-0.234083,-1.894911,0.339579,0.008465
NE,-0.000804,-0.196830,-1.707775,0.249787,-0.010020
WI,-0.041345,-0.100480,0.042776,0.045792,-0.033441
